Through this program, we wish to solve the N Queens problem using the SAT Solver based on $pycosat$ in order to perform a comparative analysis between SAT, Mixed Integer Programming and Constraint Programming.

But what is N Queens? The N Queens problem is to place $n$ queens in such a manner on an $n$ x $n$ chessboard that no queens attack each other by being in the same row, column or diagonal. 

A similar kind of problem can be formulated with the Knights as well. 

In [126]:
import pycosat 
from typing import List

In [127]:
class NQueensSAT:
    pass

$$x_{ij} = \text{True iff the tile at row } i \text{, column } j \text{ contains a queen} $$

In [128]:
def x(self, i:int, j:int) -> int:
    n = self.n
    return j+1 + (n)*i
NQueensSAT.x = x

With our board defined through the above function, we can begin to set our constraints. 

We shall start by implementing the 'Exactly One' constraint on each row of the board, to ensure only one cell through the row shall be true, signifying one queen. 

In [129]:
def exactly_one_row(self) -> List[List[int]]:
    n = self.n
    x = self.x
    clauses = []

    for r in range(n):
        clauses.append([x(r,c) for c in range(n)])
    
    for r in range(n):
        for c1 in range(n-1):
            for c2 in range(c1+1, n):    
                clauses.append([-x(r,c1), -x(r,c2)])

    return clauses

NQueensSAT.exactly_one_row = exactly_one_row

We then repeat the same for each column of the board, to ensure only one cell through the column shall be true, signifying one queen. 

In [130]:
def exactly_one_column(self) -> List[List[int]]:
    n = self.n
    x = self.x
    clauses = []

    for c in range(n):
        clauses.append([x(r,c) for r in range(n)])
    
    for c in range(n):
        for r1 in range(n-1):
            for r2 in range(r1+1, n):
                clauses.append([-x(r1,c), -x(r2,c)])

    return clauses

NQueensSAT.exactly_one_column = exactly_one_column

Finally, with the diagonals, we add the 'At Most One' constraint through clauses to ensure there is only 0 or 1 queen in each. $4n-6$ diagonals are considered, as corners are ignored. That is because they are covered under the row and column constraints.  

In [131]:
def at_most_one_diagonal(self) -> List[List[int]]:
    n = self.n
    x = self.x
    clauses = []
    for r in range(n-1):
        c1 = 0
        c2 = n-1
        for offset1 in range(0, n-r-1):
            for offset2 in range(offset1 + 1, n-r):
                clauses.append([-x(r+offset1, c1+offset1), -x(r+offset2, c1+offset2)])
                clauses.append([-x(r+offset1, c2-offset1), -x(r+offset2, c2-offset2)])
        
    for r in range(1, n-1):
        c1 = 0
        c2 = n-1
        for offset1 in range(0, r):
            for offset2 in range(offset1 + 1, r+1):
                clauses.append([-x(r-offset1, c1+offset1), -x(r-offset2, c1+offset2)])
                clauses.append([-x(r-offset1, c2-offset1), -x(r-offset2, c2-offset2)])
                
    return clauses

NQueensSAT.at_most_one_diagonal = at_most_one_diagonal

Here we call all the functions and create the solver. 

In [132]:
def construct_solver(self, n):
    self.n = n
    self.row_clauses = self.exactly_one_row()
    self.column_clauses = self.exactly_one_column()
    self.diagonal_clauses = self.at_most_one_diagonal()

NQueensSAT.__init__ = construct_solver

Here we build the CNF through the list of clauses we've created. 

In [133]:
def build_cnf(self) -> List[List[int]]:
    clauses = self.row_clauses + self.column_clauses + self.diagonal_clauses

    return clauses

NQueensSAT.build_cnf = build_cnf

Here, we assign all the positive literals to 1 on the chessboard, by identifying their cell location through the process below. Each positive literal denotes the presence of a queena t that particular location.

In [134]:
def assignment_to_board(self, literals : List[int]) -> List[List[int]]:
    solved_board = [[0] *self.n for _ in range(self.n)]
    for l in literals:
        if l > 0:
            l = l-1
            row = l // self.n
            col = l % self.n
            solved_board[row][col] = 1

    return solved_board

NQueensSAT.assignment_to_board = assignment_to_board

With the constraints set, we now have 3 functions to calculate the new coordinates for each cell of the chess board after we perform reflections about the horizontal, vertical or the diagonal axes. 

The self.x function converts the new coordinates back into their respective literal.

In [135]:
def reflect_horizontal(self, literal):
    # Mirror is to the right
    i = (literal-1) // self.n
    j = (literal-1) % self.n
    return self.x(i, self.n - j - 1)

NQueensSAT.reflect_horizontal = reflect_horizontal

In [136]:
def reflect_vertical(self, literal):   
    # Mirror is to the bottom
    i = (literal-1) // self.n
    j = (literal-1) % self.n
    return self.x(self.n - i - 1, j)

NQueensSAT.reflect_vertical = reflect_vertical

In [137]:
def reflect_diagonal(self, literal):
    # Mirror passes across a diagonal (Top left to bottom right)
    i = (literal-1) // self.n
    j = (literal-1) % self.n
    return self.x(j, i) 

NQueensSAT.reflect_diagonal = reflect_diagonal

Here, we run the solver while it continues to return optimal solutions, all the while incrementing the counter of the number of fundamental solutions found. 

A fundamental solution is one that can be rotated and reflected to create other derivative solutions. In order to improve the runtime of the code, we have decided to calculate only these fundamental solutions. 

This is done in the following manner: 
We have 3 types of transforms (reflection about the 3 axes), each of which functions as a single bit in a 3-bit binary number. From none, to all can be applied on a fundamental solution to obtain other derivative solutions. Thus, through the code below, we constrain the solver to ingore each of these solutions. 

There is an approximate 8x time saving through the above heuristic, as each fundamental solution can have a maximum of 8 different derivative solutions. 

In [138]:
from timeit import default_timer as timer
def solve(self, print = False) -> List[List[int]]:
    t = timer()

    cnf = self.build_cnf()
    literals = pycosat.solve(cnf)
    if (literals == "UNSAT"):
        raise ValueError('Impossible N Queens puzzle!')

    num_soln = 0
    while (literals != "UNSAT"):
        self.solved_board = self.assignment_to_board(literals)

        # IF PRINT IS SET TO TRUE ABOVE, ALL SOLUTIONS WILL BE PRINTED
        if print is True: 
            self.pretty_print()
        for x in range(8):
            clause = []
            for l in literals:
                if l > 0:
                    new_l = l
                    if (x&4 == 4):
                        new_l = self.reflect_diagonal(new_l)
                    if (x&2 == 2):
                        new_l = self.reflect_vertical(new_l)
                    if (x&1 == 1):
                        new_l = self.reflect_horizontal(new_l)
                    clause.append(-new_l)   
            cnf.append(clause)
        num_soln += 1
        literals = pycosat.solve(cnf)

    time_taken = timer()-t
    
    return num_soln, time_taken
    
NQueensSAT.solve = solve

A function to pretty print a single solution. 

In [139]:
def pretty_print(self):
    n = self.n
    
    # "pretty" printing the solution
    for i in range(n):
        for j in range(n):
            # print(j.solution_value())
            if self.solved_board[i][j] == 1:
                # There is a queen in column j, row i.
                print('Q', end=' ')
            else:
                print('_', end=' ')
        print()
    print()

NQueensSAT.pretty_print = pretty_print

Cell below is to find the number of fundamental solutions for $n$ in the range shown. 

In [140]:
for n in range(4, 16):
    solver = NQueensSAT(n)
    output = solver.solve()
    print('It took ' + str(output[1]) + ' seconds to find ' + str(output[0]) + ' fundamental solutions for n = ' + str(n)) 

It took 0.00031389899959322065 seconds to find 1 fundamental solutions for n = 4
It took 0.0013140409973857459 seconds to find 2 fundamental solutions for n = 5
It took 0.00092125599985593 seconds to find 1 fundamental solutions for n = 6
It took 0.003323616001580376 seconds to find 6 fundamental solutions for n = 7
It took 0.012813851997634629 seconds to find 12 fundamental solutions for n = 8
It took 0.09176884500266169 seconds to find 46 fundamental solutions for n = 9
It took 0.4920597029995406 seconds to find 92 fundamental solutions for n = 10
It took 4.393774369997118 seconds to find 341 fundamental solutions for n = 11
It took 147.75208694499815 seconds to find 1787 fundamental solutions for n = 12


KeyboardInterrupt: 